In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from os.path import join as oj
import sys
sys.path.append('../src')
import numpy as np
import seaborn as sns
import torch
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
plt.style.use('dark_background')
import data
from skorch.callbacks import Checkpoint
from skorch import NeuralNetRegressor
from config import *
from tqdm import tqdm
import pickle as pkl
import train_reg
from copy import deepcopy
import config
import models
import pandas as pd
import features
from scipy.stats import skew, pearsonr
import outcomes
import neural_networks
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.nn import functional as F
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import confusion_matrix

# prepare data

In [2]:
dsets = ['clath_aux+gak_a7d2', 'clath_aux+gak', 'clath_aux+gak_a7d2_new', 'clath_aux+gak_new', 'clath_gak']
splits = ['train', 'test']
#feat_names = ['X_same_length_normalized'] + data.select_final_feats(data.get_feature_names(df))
              #['mean_total_displacement', 'mean_square_displacement', 'lifetime']
meta = ['cell_num', 'Y_sig_mean', 'Y_sig_mean_normalized']
dfs = data.load_dfs_for_lstm(dsets=dsets, splits=splits, meta=meta)

100%|██████████| 5/5 [00:51<00:00, 10.28s/it]


**analyze cv results**

In [23]:
out_dir = f'{DIR_RESULTS}/dec13_deep_best'
results = train_reg.load_results(out_dir)
r = results
r = r[[k for k in r if not 'std' in k]]
r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
r = r.sort_values(by=['r2'], ascending=False)
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
r[['pearsonr', 'r2']]

{'r2': [0.3410249004640089, 0.3208547273943092, 0.4126897499403642, 0.3594438570984312, 0.36538168636939417, 0.408612136123226, 0.3859741881617027, 0.4261156649998624], 'pearsonr': [0.5854548719270554, 0.56699047930762, 0.6433357214073335, 0.601086041946158, 0.6045239439241363, 0.6392560386371637, 0.6236078945096376, 0.6549200084527723]}
dict_keys(['r2', 'pearsonr'])


,pearsonr,r2
model_type,,
clath_aux+gak_a7d2_new_Y_sig_mean_normalized_nn_lstm_100_40,0.615,0.377


# transfer to classification

In [73]:
results = pkl.load(open('../models/dnn_full_long_normalized_across_track_1_feat.pkl', 'rb'))
# results = pkl.load(open('../models/clath_aux+gak_a7d2_new_Y_sig_mean_normalized_nn_lstm_100_40.pkl', 'rb'))
dnn = neural_networks.neural_net_sklearn(D_in=40, H=20, p=0, arch='lstm')
dnn.model.load_state_dict(results['model_state_dict'])

<All keys matched successfully>

In [ ]:
R = 2
C = len(dfs) // 2

# exclude the training data
ds = {(k, v): dfs[(k, v)]
      for (k, v) in sorted(dfs.keys(), key=lambda x: x[1] + x[0])
      #if not k == 'clath_aux+gak_a7d2_new'
     }


plt.figure(dpi=200, figsize=(C * 3, R * 3))
for i, (k, v) in enumerate(ds.keys()):
    df = ds[(k, v)]
    X = df[feat_names[:1]]
    y = df['y_consec_thresh']
    preds = dnn.predict(X)
    
    plt.subplot(R, C, i + 1)
    y_pred = (preds > 0).astype(int)
    acc = np.mean(y_pred == y)
    print(k, v, np.mean(y), acc)
    plt.title(f'{k} {v} {100*acc:0.1f}%', fontsize=10)
    plt.hist(preds[y==0], alpha=1, label='aux-')
    plt.hist(preds[y==1], alpha=0.5, label='aux+')
    plt.yscale('log')
    plt.xlabel('pred')
    
    if i == 0:
        plt.legend()

In [ ]:
R = 2
C = len(dfs) // 2 

plt.figure(dpi=200, figsize=(C * 3, R * 3))
for i, (k, v) in enumerate(ds.keys()):
    df = ds[(k, v)]
    X = df[feat_names]
    y = df['Y_sig_mean_normalized']
    preds = dnn.predict(X)
    
    plt.subplot(R, C, i + 1)
    acc = np.corrcoef(preds, y)[0, 1]
    print(k, v) #, np.corrcoef(preds, y))
    plt.title(f'{k} {v} {100*acc:0.1f}', fontsize=10)    
    plt.plot(preds, df['Y_sig_mean_normalized'], '.')
    plt.xlabel('pred')
    plt.ylabel('aux_sig_mean')
plt.show()

# plot tracks where lstm does better than gb

In [5]:
results = pkl.load(open('../models/dnn_full_long_normalized_across_track_1_feat.pkl', 'rb'))
# results = pkl.load(open('../models/clath_aux+gak_a7d2_new_Y_sig_mean_normalized_nn_lstm_100_40.pkl', 'rb'))
dnn = neural_networks.neural_net_sklearn(D_in=40, H=20, p=0, arch='lstm')
dnn.model.load_state_dict(results['model_state_dict'])
gb = GradientBoostingRegressor()
gb.fit(df_full[feat_names[1:]], df_full['Y_sig_mean_normalized'].values)

GradientBoostingRegressor()

In [32]:
df = ds[('clath_aux+gak_a7d2', 'test')]
X = df[feat_names[:1]]
y = df['y_consec_thresh']
preds_dnn = dnn.predict(X)
preds_gb = gb.predict(df[feat_names[1:]])

In [57]:
errs_gb = (preds_gb > 0) != y
errs_dnn = (preds_dnn > 0) != y
idxs_only_gb_errs = errs_gb & (~errs_dnn)
diffs = np.abs(preds_dnn - preds_gb)
idxs_diff_biggest = np.argsort(diffs)[::-1]
idxs_only_gb_errs = idxs_only_gb_errs.values[idxs_diff_biggest]
diffs = diffs[idxs_diff_biggest]

In [ ]:
R = 4
C = 7
d = df[idxs_only_gb_errs]
plt.figure(dpi=50, figsize=(C * 4, R * 4))
for i in range(R * C):
    plt.subplot(R, C, i + 1)
    plt.plot(df['X_same_length'].iloc[i], color='red')
    plt.xticks([])
    plt.title(f'diff: {diffs[i]:.2f}')
#     plt.title(f"y:{y.values[dnn_best[i]]:0.1f}, gb:{preds_gb[dnn_best[i]]:0.1f}, dnn:{preds_dnn[dnn_best[i]]:0.1f}")
    
plt.tight_layout()
plt.show()

In [ ]:
plt.scatter(preds_dnn, preds_gb)
plt.xlabel("prediction of lstm")
plt.ylabel("prediction of gradient boosting")